In [41]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime,date
import glob
from pprint import pprint

In [2]:
# parentDir = os.path.abspath(os.path.join(os.getcwd(),'..'))
# os.chdir(parentDir)

# Check data_loader

## Check load_transaction

In [43]:


def load_transaction(data_folder_path, transaction_file_pattern):
    transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
    transactions = _combine_transaction_files(transaction_files)
    transactions = _clean_transactions(transactions)
    print(f"The latest transaction date is {transactions['Run Date'].max()}")
    return transactions


def _gather_transaction_files(data_folder_path,transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)
    return transaction_files

def _combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(14)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions

def _remove_NA_value(df,colName):
    df_copy = df.copy()
    df_copy = df_copy[
        df_copy[colName].notna()
    ] 
    return df_copy

def _remove_leading_space(df,colName):
    df_copy = df.copy()
    df_copy[colName] = df_copy[colName].str.lstrip()
    return df_copy

def _str_to_date(df, colName, format):
    df_copy = df.copy()
    df_copy[colName] = pd.to_datetime(
        df_copy[colName], format=format
    ).dt.date
    return df_copy

def _add_Transfer_symbol(df):
    df_copy = df.copy()
    df_copy.loc[df_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    return df_copy

def _sort_df_by_column(df, colName):
    df_copy = df.copy()
    df_copy = df_copy.sort_values(by=colName).reset_index(
        drop=True
    )
    return df_copy


def _clean_transactions(transactions):
    transactions = _remove_NA_value(transactions,"Amount ($)")
    transactions = _remove_leading_space(transactions,"Run Date")
    transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
    transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
    transactions = _add_Transfer_symbol(transactions)
    transactions = _remove_leading_space(transactions,"Symbol")
    transactions = _remove_leading_space(transactions,"Description")
    transactions = _sort_df_by_column(transactions,"Run Date")
    return transactions


In [44]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [45]:
transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
transactions = _combine_transaction_files(transaction_files)
transactions = _clean_transactions(transactions)

In [46]:
# transactions = _remove_NA_value(transactions,"Amount ($)")
# transactions = _remove_leading_space(transactions,"Run Date")
# transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
# transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
# transactions = _add_Transfer_symbol(transactions)
# transactions = _remove_leading_space(transactions,"Symbol")
# transactions = _sort_df_by_column(transactions,"Run Date")

## Check load_position

In [271]:

def load_position(data_folder_path, position_file_pattern):
    position_file = _gather_position_files(data_folder_path, position_file_pattern)

    if not position_file is None:
        position = pd.read_csv(position_file)
        position = _clean_position(position)
    else:
        print("No position file found.")
    return position

def _gather_position_files(data_folder_path, position_file_pattern):
    position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
    position_files = glob.glob(position_file_path_pattern)
    position_file = _find_latest_position_file(position_files)
    return position_file


def _find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file


def _clean_position(position):
    position = _remove_NA_value(position,"Current Value")
    position = _transfer_dollar_to_float(position, "Current Value")
    position = _transfer_dollar_to_float(position, "Cost Basis Total")
    return position


def _transfer_dollar_to_float(df, colNames):
    """
    Change "$123,456" to 123456.0, and "--" to 0.0
    """
    df_copy = df.copy()
    # Replace any "--" with "$0"
    cleaned = df_copy[colNames].str.replace("--", "$0", regex=False)
    # Remove dollar sign and commas, then convert to float
    cleaned = cleaned.str.replace("$", "", regex=False).str.replace(",", "", regex=False)
    df_copy[colNames] = cleaned.astype(float)
    return df_copy


In [48]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [49]:
position_file = _gather_position_files(data_folder_path, position_file_pattern)
position_file

'./data/Portfolio_Positions_Aug-05-2025.csv'

In [50]:
position = pd.read_csv(position_file)
position = _clean_position(position)

In [51]:
# position = _remove_NA_value(position,"Current Value")
# position = _transfer_dollar_to_float(position, "Current Value")
# position = _transfer_dollar_to_float(position, "Cost Basis Total")

# Check Portfolio

In [52]:
from scipy.optimize import newton
import numpy as np
def compute_irr(cashflows, dates, cob):
    """用 Newton-Raphson 方法计算 IRR（连续复利）"""
    def npv(r):
        return sum(cf * np.exp(-r * (cob - d).days / 365.0) for cf, d in zip(cashflows, dates))
    try:
        result = -newton(npv, 0.1)
    except RuntimeError:
        result = np.nan
    return result

def display_percentage(df, colNames: list):
    df_copy = df.copy()
    for col in colNames:
        df_copy[col] = df_copy[col].apply(lambda x: f"{x:.2%}")
    return df_copy

In [ ]:
class Portfolio:
    def __init__(self, transactions, position):
        self.transactions = transactions
        self.position = position
        self.cob = date.today()
        
        self.account_number_dic = {
            "Individual":'Z23390746',
            "401k":'86964',
            "HSA":'241802439',
            "Cash":'Z06872898'
        }
        
        self.pensionTransactions = transactions[transactions['Account Number']==self.account_number_dic["401k"]]
        self.pensionPosition = position[position['Account Number']==self.account_number_dic["401k"]]
        self.HSATransactions = transactions[transactions['Account Number']==self.account_number_dic["HSA"]]
        self.HSAPosition = position[position['Account Number']==self.account_number_dic["HSA"]]
        self.cashTransactions = transactions[transactions['Account Number']==self.account_number_dic["Cash"]]
        self.cashPosition = position[position['Account Number']==self.account_number_dic["Cash"]]
        
        
        self.cashSymbols = ['FZFXX**','FZFXX']
        self.otherSymbols = ['Pending Activity']
        self.contributionSymbols = ['','Transfer']
        self.bondSymbols = self.get_bond_symbol_list()
        self.stockSymbols = self.get_stock_symbol_list()
        
        self.cashDescriptions = ['HELD IN MONEY MARKET','FIDELITY TREASURY MONEY MARKET FUND']
        self.otherDescriptions = []
        self.contributionDescriptions = ['No Description']
        self.bondDescriptions = self.get_bond_description_list()
        self.stockDescriptions = self.get_stock_description_list()
        
    ## Main functions
    
    def get_account_summary(self):
        colName = 'Symbol'
        totalValueBond, totalIrrBond, holdingPeriodBond = self.get_combined_bond_result(colName)
        totalValueSymbol, totalIrrSymbol, holdingPeriodSymbol = self.get_combined_stock_result(colName)
        totalValueCash, totalIrrCash, holdingPeriodCash = self.get_combined_cash_result(colName)
        
        
        totalIRR = self.get_total_irr(colName)
        totalValue = self.position['Current Value'].sum()
        totalHoldingPeriod = (totalValueBond*holdingPeriodBond+totalValueSymbol*holdingPeriodSymbol+totalValueCash*holdingPeriodCash)/(totalValueBond+totalValueSymbol+totalValueCash)
        
        
        result = pd.DataFrame({
            'Type':['bond','stock','cash','Total'],
            'Value': [totalValueBond,totalValueSymbol,totalValueCash,totalValue],
            'Percentage':[totalValueBond,totalValueSymbol,totalValueCash,totalValue]/totalValue,
            'IRR':[totalIrrBond, totalIrrSymbol, totalIrrCash,totalIRR],
            'Weighted Avg Holding Period':[holdingPeriodBond,holdingPeriodSymbol,holdingPeriodCash,totalHoldingPeriod]
        } 
        )
        return result
    
    def get_all_bond_summary(self, colName='Symbol',showExpiredBond=False):
        listKeys = self.get_bond_key_list(colName)
        currentValueResult = self.get_key_current_values(listKeys,colName)
        irrResult = self.get_key_irrs(listKeys,colName)
        holdingPeriodResult = self.get_key_holding_period(listKeys,colName)
        result = pd.merge(currentValueResult, irrResult, on=colName)
        result = pd.merge(result, holdingPeriodResult, on=colName)
        result = result.sort_values(by='Current Value', ascending=False)
        if not showExpiredBond:
            result = result[result['Current Value']>0]
        return result
        
    def get_all_stock_summary(self, colName='Symbol'):
        listKeys = self.get_stock_key_list(colName)
        currentValueResult = self.get_key_current_values(listKeys,colName)
        irrResult = self.get_key_irrs(listKeys,colName)
        holdingPeriodResult = self.get_key_holding_period(listKeys,colName)
        result = pd.merge(currentValueResult, irrResult, on=colName)
        result = pd.merge(result, holdingPeriodResult, on=colName)
        result = result.sort_values(by='Current Value', ascending=False)
        return result
    
    
    
    
    ## Help functions
    
    def get_combined_bond_result(self, colName):
        listKeys = self.get_bond_key_list(colName)
        totalValue = self.get_total_keys_value(listKeys,colName)
        totalIrr = self.get_combined_key_irr(listKeys,colName)
        holdingPeriod = self.get_combined_key_holding_period(listKeys,colName)
        return totalValue, totalIrr, holdingPeriod
    
    def get_combined_stock_result(self, colName):
        listKeys = self.get_stock_key_list(colName)
        totalValue = self.get_total_keys_value(listKeys,colName)
        totalIrr = self.get_combined_key_irr(listKeys,colName)
        holdingPeriod = self.get_combined_key_holding_period(listKeys,colName)
        return totalValue, totalIrr, holdingPeriod
    
    def get_combined_cash_result(self, colName):
        listKeys = self.get_cash_key_list(colName)
        totalValue = self.get_total_keys_value(listKeys,colName)
        totalIrr = self.get_combined_key_irr(listKeys,colName)
        holdingPeriod = self.get_combined_key_holding_period(listKeys,colName)
        return totalValue, totalIrr, holdingPeriod
    
    
    
    
    def get_combined_key_holding_period(self, listKeys: list, colName, unit = 30):
        subTransactions = self.transactions[self.transactions[colName].isin(listKeys)]
        buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
        df = buyTranactions.copy()
        df['Days Held'] = (self.cob - df['Run Date']).apply(lambda x: x.days)
        df['Weight'] = df['Amount ($)'].abs()
        totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
        return totalWeightedHold
        
    def get_key_holding_period(self, listKeys: list, colName, unit = 30):
        subTransactions = self.transactions[self.transactions[colName].isin(listKeys)]
        buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
        df = buyTranactions.copy()
        df['Days Held'] = (self.cob - df['Run Date']).apply(lambda x: x.days)
        df['Weight'] = df['Amount ($)'].abs()
        totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
        totalWeightedHoldRow = pd.DataFrame({colName: ['Total'], 'Weighted Avg Holding Period': [totalWeightedHold]})

        # 分组计算加权平均
        weightedHold = (
            df.groupby(colName)
            .apply(lambda g: (g['Days Held'] * g['Weight']).sum() / g['Weight'].sum()/ unit, include_groups=False)
            .reset_index(name='Weighted Avg Holding Period')
        )
        
        weightedHold = pd.concat([weightedHold, totalWeightedHoldRow], ignore_index=True)
        return weightedHold
    
    
    def get_key_current_values(self, listKeys: list, colName):
        resultList = []
        totalCurrentValue = self.get_total_keys_value(listKeys, colName)
        for key in listKeys:
            currentValue = self.get_key_current_value(key, colName)
            currentValuePercent = currentValue/totalCurrentValue
            resultList.append({
                colName: key,
                'Current Value': currentValue,
                'Percentage': currentValuePercent
            })
        resultList.append({
                colName: 'Total',
                'Current Value': totalCurrentValue,
                'Percentage': 1
            })
        return pd.DataFrame(resultList)
    
    def get_key_current_value(self,key, colName):
        try:
            value = self.position.loc[self.position[colName]==key, 'Current Value'].values[0]
        except:
            value = 0
        return value
        
    def get_total_keys_value(self, listKeys: list, colName):
        Position = self.position[self.position[colName].isin(listKeys)]
        totalValue = Position['Current Value'].sum()
        return totalValue
    
    def get_key_irrs(self, listKeys: list, colName):
        resultList = []
        for key in listKeys:
            irr = self.get_combined_key_irr([key], colName)
            resultList.append({
                colName: key,
                'IRR': irr
            })
        totalIrr = self.get_combined_key_irr(listKeys, colName)
        resultList.append({
                colName: 'Total',
                'IRR': totalIrr
            })
        return pd.DataFrame(resultList)
    
    def get_total_irr(self, colName):
        if colName == 'Symbol':
            contributionKeys = self.contributionSymbols
        elif colName == 'Description':
            contributionKeys = self.contributionDescriptions
        else:
            return 0
        trans = self.transactions[self.transactions[colName].isin(contributionKeys)]
        cashflows = trans['Amount ($)'].tolist()
        cashflows = [-x for x in cashflows]
        dates = trans['Run Date'].tolist()
        current_value = self.position['Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    
    
    def get_combined_key_irr(self, listKeys: list, colName):
        trans = self.transactions[self.transactions[colName].isin(listKeys)]
        cashflows = trans['Amount ($)'].tolist()
        dates = trans['Run Date'].tolist()
        current_value = self.position.loc[self.position[colName].isin(listKeys), 'Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    
    
    def get_stock_key_list(self, colName='Description'):
        if colName == 'Symbol':
            listKeys = self.get_stock_symbol_list()
        elif colName == 'Description':
            listKeys = self.get_stock_description_list()
        else:
            listKeys = []
        return listKeys
    
    def get_stock_symbol_list(self):
        symbols = self.transactions['Symbol'].unique()
        stockSymbols = [
            sym for sym in symbols
            if sym not in self.cashSymbols
            and not str(sym).startswith('91')
            and sym not in self.otherSymbols
            and sym not in self.contributionSymbols
        ]
        return stockSymbols
    
    def get_stock_description_list(self):
        descriptions = self.transactions['Description'].unique()
        stockDescriptions = [
            desc for desc in descriptions
            if desc not in self.cashDescriptions
            and not str(desc).startswith('UNITED STATES TREAS BILLS')
            and desc not in self.otherDescriptions
            and desc not in self.contributionDescriptions
        ]
        return stockDescriptions
    
    def get_bond_key_list(self, colName='Description'):
        if colName == 'Symbol':
            listKeys = self.get_bond_symbol_list()
        elif colName == 'Description':
            listKeys = self.get_bond_description_list()
        else:
            listKeys = []
        return listKeys
    
    def get_bond_symbol_list(self):
        symbols = self.transactions['Symbol'].unique()
        bondSymbols = [
            sym for sym in symbols
            if  str(sym).startswith('91')
        ]
        return bondSymbols
    
    def get_bond_description_list(self):
        descriptions = self.transactions['Description'].unique()
        bondDescriptions = [
            desc for desc in descriptions
            if  str(desc).startswith('UNITED STATES TREAS BILLS')
        ]
        return bondDescriptions
    
    
    
    def get_cash_key_list(self, colName):
        if colName == 'Symbol':
            listKeys = self.cashSymbols
        elif colName == 'Description':
            listKeys = self.cashDescriptions
        else:
            listKeys = []
        return listKeys
    
    def get_401k_description_list(self):
        descriptions = self.pensionTransactions['Description'].unique()
        return descriptions
    


class IndividualPortfolio(Portfolio):
    ## BBRK's description is different for trans and posi
    def __init__(self, transactions, position):
        super().__init__(transactions, position)
        self.transactions = transactions[transactions['Account Number']==self.account_number_dic["Individual"]]
        self.position = position[position['Account Number']==self.account_number_dic["Individual"]]
        
        
class PensionPortfolio(Portfolio):
    def __init__(self, transactions, position):
        super().__init__(transactions, position)
        self.transactions = transactions[transactions['Account Number']==self.account_number_dic["401k"]]
        self.position = position[position['Account Number']==self.account_number_dic["401k"]]
        
    def get_combined_key_irr(self, listKeys: list, colName):
        trans = self.transactions[self.transactions[colName].isin(listKeys)]
        cashflows = trans['Amount ($)'].tolist()
        cashflows = [-x for x in cashflows]
        dates = trans['Run Date'].tolist()
        current_value = self.position.loc[self.position[colName].isin(listKeys), 'Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    

class HSAPortfolio(Portfolio):
    def __init__(self, transactions, position):
        super().__init__(transactions, position)
        self.transactions = transactions[transactions['Account Number']==self.account_number_dic["HSA"]]
        self.position = position[position['Account Number']==self.account_number_dic["HSA"]]
        

## Check get_individual_account_summary

In [272]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
individualPortfolio = IndividualPortfolio(transactions, position)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


### Check get_account_summary

#### Full Run

In [273]:
result = individualPortfolio.get_account_summary()
display_percentage(result,['Percentage','IRR'])

,Type,Value,Percentage,IRR,Weighted Avg Holding Period
0,bond,746743.50,62.47%,4.37%,6.113349
1,stock,423303.14,35.41%,16.30%,12.172148
2,cash,25294.19,2.12%,nan%,10.239369
3,Total,1195340.83,100.00%,8.67%,8.346246


In [274]:
result = portfolio.get_account_summary()
display_percentage(result,['Percentage','IRR'])

,Type,Value,Percentage,IRR,Weighted Avg Holding Period
0,bond,746743.50,61.16%,4.37%,6.113349
1,stock,426721.18,34.95%,20.81%,12.167167
2,cash,25294.19,2.07%,nan%,10.239369
3,Total,1220905.61,100.00%,10.44%,8.355382


#### Step Run

In [249]:
colName = 'Symbol'
print(individualPortfolio.get_combined_bond_result(colName))
print(portfolio.get_combined_bond_result(colName))

(np.float64(746743.5), np.float64(0.043702941160968276), np.float64(6.11334908346728))
(np.float64(746743.5), np.float64(0.043702941160968276), np.float64(6.11334908346728))


In [250]:
colName = 'Description'
print(individualPortfolio.get_combined_bond_result(colName))
print(portfolio.get_combined_bond_result(colName))

(np.float64(746743.5), np.float64(0.043702941160968276), np.float64(6.11334908346728))
(np.float64(746743.5), np.float64(0.043702941160968276), np.float64(6.11334908346728))


In [251]:
colName = 'Symbol'
print(individualPortfolio.get_combined_stock_result(colName))
print(portfolio.get_combined_stock_result(colName))

(np.float64(423303.14), np.float64(0.1629841014894518), np.float64(12.1721479786138))
(np.float64(426721.18), np.float64(0.20811241965775165), np.float64(12.167167164698956))


In [252]:
colName = 'Description'
print(individualPortfolio.get_combined_stock_result(colName))
print(portfolio.get_combined_stock_result(colName))

(np.float64(418641.33999999997), np.float64(0.1525249895861842), np.float64(12.1721479786138))
(np.float64(440370.6599999999), np.float64(0.23130699899900964), np.float64(12.167167164698956))


In [253]:
colName = 'Symbol'
print(individualPortfolio.get_combined_cash_result(colName))
print(portfolio.get_combined_cash_result(colName))

(np.float64(25294.19), nan, np.float64(10.239368662932643))
(np.float64(25294.19), nan, np.float64(10.239368662932643))


In [254]:
colName = 'Description'
print(individualPortfolio.get_combined_cash_result(colName))
print(portfolio.get_combined_cash_result(colName))

(np.float64(25294.19), nan, np.float64(10.239368662932643))
(np.float64(29022.399999999998), nan, np.float64(10.239368662932643))


In [ ]:
colName = 'Symbol'
if colName == 'Symbol':
    listKeys = portfolio.get_stock_symbol_list()
elif colName == 'Description':
    listKeys = portfolio.get_stock_description_list()
else:
    0
trans = portfolio.transactions[portfolio.transactions[colName].isin(listKeys)]
cashflows = trans['Amount ($)'].tolist()
dates = trans['Run Date'].tolist()
current_value = portfolio.position.loc[portfolio.position[colName].isin(listKeys), 'Current Value'].sum()
# cashflows.append(current_value)
# dates.append(portfolio.cob)

(np.float64(418641.33999999997), np.float64(0.1525249895861842), np.float64(12.1721479786138))
(np.float64(440370.6599999999), np.float64(0.23130699899900964), np.float64(12.167167164698956))


In [228]:
colNameS = 'Symbol'
listKeysS = individualPortfolio.get_stock_symbol_list()
transS = individualPortfolio.transactions[individualPortfolio.transactions[colNameS].isin(listKeysS)]
cashflowsS = transS['Amount ($)'].tolist()
datesS = transS['Run Date'].tolist()
current_valueS = individualPortfolio.position.loc[individualPortfolio.position[colNameS].isin(listKeysS), 'Current Value'].sum()
# cashflows.append(current_value)
# dates.append(portfolio.cob)

In [229]:
colNameD = 'Description'
listKeysD = individualPortfolio.get_stock_description_list()
transD = individualPortfolio.transactions[individualPortfolio.transactions[colNameD].isin(listKeysD)]
cashflowsD = transD['Amount ($)'].tolist()
datesD = transD['Run Date'].tolist()
current_valueD = individualPortfolio.position.loc[individualPortfolio.position[colNameD].isin(listKeysD), 'Current Value'].sum()
# cashflows.append(current_value)
# dates.append(portfolio.cob)

In [230]:
[current_valueS,current_valueD]

[np.float64(423303.14), np.float64(418641.33999999997)]

### Check get_all_stock_summary

#### Full Run

In [332]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
individualPortfolio = IndividualPortfolio(transactions, position)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [334]:
result = individualPortfolio.get_all_stock_summary()
display_percentage(result,['Percentage','IRR'])

,Symbol,Current Value,Percentage,IRR,Weighted Avg Holding Period
21,Total,423303.14,100.00%,16.30%,12.172148
8,FXAIX,163865.08,38.71%,19.73%,9.473456
11,FSKAX,113381.14,26.78%,13.44%,11.276006
10,FSPSX,69497.82,16.42%,13.14%,13.441002
13,MSFT,15867.00,3.75%,16.98%,12.891512
0,AAPL,14233.47,3.36%,10.24%,34.997598
7,TSLA,6193.40,1.46%,32.63%,18.534063
9,MCD,6013.60,1.42%,14.87%,14.835822
14,BRKB,4661.80,1.10%,1.69%,9.600000
4,GOOGL,3888.10,0.92%,30.96%,29.867213


#### Step Run

##### Current Value

In [287]:
colName='Description'

In [294]:
listKeys = individualPortfolio.get_stock_key_list(colName)
currentValueResult = individualPortfolio.get_key_current_values(listKeys,colName)
irrResult = individualPortfolio.get_key_irrs(listKeys,colName)
holdingPeriodResult = individualPortfolio.get_key_holding_period(listKeys,colName)

In [302]:
unit = 30
subTransactions = individualPortfolio.transactions[individualPortfolio.transactions[colName].isin(listKeys)]
buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
df = buyTranactions.copy()
df['Days Held'] = (individualPortfolio.cob - df['Run Date']).apply(lambda x: x.days)
df['Weight'] = df['Amount ($)'].abs()
totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
totalWeightedHoldRow = pd.DataFrame({colName: ['Total'], 'Weighted Avg Holding Period': [totalWeightedHold]})


In [307]:
weightedHold = (
            df.groupby(colName)
            .apply(lambda g: (g['Days Held'] * g['Weight']).sum() / g['Weight'].sum()/ unit, include_groups=False)
            .reset_index(name='Weighted Avg Holding Period')
        )
weightedHold.shape

(21, 2)

In [308]:
weightedHold = (
            df.groupby('Symbol')
            .apply(lambda g: (g['Days Held'] * g['Weight']).sum() / g['Weight'].sum()/ unit, include_groups=False)
            .reset_index(name='Weighted Avg Holding Period')
        )
weightedHold.shape

(21, 2)

In [ ]:
stockSymbols = portfolio.get_stock_symbol_list()
currentValueResult = portfolio.get_key_current_values(stockSymbols,'Symbol').head(5)
currentValueResult.head(5)

,Key,Current Value,Percentage
0,AAPL,14233.47,0.033625
1,SBUX,2707.05,0.006395
2,JPM,2915.42,0.006887
3,AXP,2981.20,0.007043
4,GOOGL,3888.10,0.009185


In [121]:
stockDescriptions = portfolio.get_stock_description_list()
portfolio.get_key_current_values(stockDescriptions,'Description').head(5)

,Key,Current Value,Percentage
0,APPLE INC,14233.47,0.033999
1,STARBUCKS CORP COM USD0.001,2707.05,0.006466
2,JPMORGAN CHASE &CO. COM,2915.42,0.006964
3,AMERICAN EXPRESS CO COM USD0.20,2981.20,0.007121
4,ALPHABET INC CAP STK CL A,3888.10,0.009287


##### IRR

In [ ]:
irrResult = portfolio.get_key_irrs(stockSymbols, 'Symbol')
irrResult.head(5)

,Key,IRR
0,AAPL,0.102383
1,SBUX,0.065357
2,JPM,0.323283
3,AXP,0.245696
4,GOOGL,0.309569
5,AMZN,0.329863
6,NKE,-0.071231
7,TSLA,0.326300
8,FXAIX,0.195717
9,MCD,0.148729


In [89]:
holdingPeriod = portfolio.get_symbol_holding_period(stockSymbols)
holdingPeriod.head(5)

,Symbol,Weighted Avg Holding Period
0,AAPL,34.997598
1,AMZN,29.900000
2,AXP,35.666667
3,BRKB,9.600000
4,COKE,13.218008


,Key,Current Value,Percentage
0,APPLE INC,14233.47,0.033999
1,STARBUCKS CORP COM USD0.001,2707.05,0.006466
2,JPMORGAN CHASE &CO. COM,2915.42,0.006964
3,AMERICAN EXPRESS CO COM USD0.20,2981.20,0.007121
4,ALPHABET INC CAP STK CL A,3888.10,0.009287


In [ ]:
listKeys = stockDescriptions
colName = 'Description'

In [ ]:
resultList = []
totalCurrentValue = portfolio.get_total_keys_value(listKeys, colName)
totalCurrentValue

np.float64(418641.33999999997)

In [ ]:
key = listKeys[0]
print(key)
currentValue = portfolio.get_key_current_value(key)
currentValue

APPLE INC


0

### Check get_all_bond_summary

In [329]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
individualPortfolio = IndividualPortfolio(transactions, position)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [331]:
result = individualPortfolio.get_all_bond_summary(colName='Symbol',showExpiredBond=False)
display_percentage(result,['Percentage','IRR'])

,Symbol,Current Value,Percentage,IRR,Weighted Avg Holding Period
54,Total,746743.5,100.00%,4.37%,6.113349
46,912797PW1,149338.5,20.00%,3.71%,0.820074
50,912797QU4,149253.0,19.99%,3.37%,0.899616
48,912797MG9,99976.0,13.39%,4.18%,1.433333
49,912797QL4,99751.0,13.36%,3.88%,1.366667
51,912797QT7,99586.0,13.34%,3.41%,0.900000
53,912797QW0,99336.0,13.30%,0.20%,0.200000
52,912797NA1,49503.0,6.63%,3.14%,0.300000


## Check get 401k account summary

In [371]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
pensionPortfolio = PensionPortfolio(transactions, position)

The latest transaction date is 2025-08-05


In [373]:
pensionPortfolio.get_all_stock_summary(colName='Description')

,Description,Current Value,Percentage,IRR,Weighted Avg Holding Period
2,Total,18311.28,1.000000,0.193962,8.769103
1,SP 500 INDEX PL CL E,12580.24,0.687021,0.170868,8.755419
0,FID BLUE CHIP GR K6,5731.04,0.312979,0.245646,8.800821


In [362]:
colName='Description'

In [366]:
listKeys = pensionPortfolio.get_stock_key_list(colName)
currentValueResult = pensionPortfolio.get_key_current_values(listKeys,colName)
currentValueResult

,Description,Current Value,Percentage
0,FID BLUE CHIP GR K6,5731.04,0.312979
1,SP 500 INDEX PL CL E,12580.24,0.687021
2,Total,18311.28,1.000000


In [367]:
irrResult = pensionPortfolio.get_key_irrs(listKeys,colName)
irrResult

,Description,IRR
0,FID BLUE CHIP GR K6,0.245646
1,SP 500 INDEX PL CL E,0.170868
2,Total,0.193962


In [365]:
holdingPeriodResult = pensionPortfolio.get_key_holding_period(listKeys,colName)
holdingPeriodResult

,Description,Weighted Avg Holding Period
0,FID BLUE CHIP GR K6,8.800821
1,SP 500 INDEX PL CL E,8.755419
2,Total,8.769103


In [369]:
result = pd.merge(currentValueResult, irrResult, on=colName)
result = pd.merge(result, holdingPeriodResult, on=colName)
result = result.sort_values(by='Current Value', ascending=False)
result

,Description,Current Value,Percentage,IRR,Weighted Avg Holding Period
2,Total,18311.28,1.000000,0.193962,8.769103
1,SP 500 INDEX PL CL E,12580.24,0.687021,0.170868,8.755419
0,FID BLUE CHIP GR K6,5731.04,0.312979,0.245646,8.800821


In [354]:
key = listKeys[0]
irr = pensionPortfolio.get_combined_key_irr([key], colName)
irr

nan

In [357]:
trans = pensionPortfolio.transactions[pensionPortfolio.transactions[colName].isin(listKeys)]
current_value = pensionPortfolio.position.loc[pensionPortfolio.position[colName].isin(listKeys), 'Current Value'].sum()

In [358]:
current_value

np.float64(18311.28)